In [1]:
#https://towardsdatascience.com/implementing-pca-from-scratch-ea3970714d2b
class PCA_impl:
    def __init__(self,n_components,random_state=None,standardize=True):
        self.n_components = n_components
        self.random_state = random_state
        self.standardize = standardize

    def fit(self,X):

        self.X_copy = X.copy().astype('float32')

        if self.standardize==True:
            self.mean_ = np.mean(self.X_copy, axis=0)
            self.std_ = np.std(self.X_copy, axis=0)
            self.X_copy -= self.mean_
            self.X_copy /= self.std_

        if self.random_state ==True:
            random.seed(0)

        cov_mat = np.cov(self.X_copy.T)    
        eigen_values, eigen_vectors = np.linalg.eig(cov_mat)
        sorted_index = np.argsort(eigen_values)[::-1]
        self.sorted_eigenvalue_ = eigen_values[sorted_index]
        self.sorted_eigenvectors_ = eigen_vectors[:,sorted_index]
        self.explained_variance_ = self.sorted_eigenvalue_[0:self.n_components]
        total_var = self.sorted_eigenvalue_.sum()
        self.explained_variance_ratio_ = self.explained_variance_ / total_var
        self.cum_var_explained = np.cumsum(self.explained_variance_ratio_)

        self.projection_matrix_ = self.sorted_eigenvectors_[:,0:self.n_components]
        
        return

    def fit_transform(self,X,y):
        self.fit(X)
        self.components_ = np.dot(self.X_copy,self.projection_matrix_)
        self.df = pd.DataFrame(data=self.components_, columns=['PC{}'.format(i+1) for i in range(self.n_components)])
        self.df['label'] = y
        return self.components_

    def pca_plot2d(self):
        fig = px.scatter(self.components_, x=0, y=1, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show()

    def pca_plot3d(self):
        fig = px.scatter_3d(self.components_, x=0, y=1,z=2, color=self.df.label,labels={'0': 'PC 1', '1': 'PC 2'})
        fig.show() 